# 理论知识查漏补缺

## 判别模型与生成模型

- 判别模型：直接对$P(y|x)$建模
- 生成模型：判断$P(y=0|x)$和$P(y=1|x)$谁更大一点，借助贝叶斯公式，对$P(x,y)$联合概率进行建模

## 高斯判别分析

## 朴素贝叶斯

动机：简化运算



$$\hat{y} = arg\max\limits_{y}P(x|y)P(y)$$
### 对于先验P(y)
- 二分类：$y\sim Bernoulli$
- 多分类：$y\sim Categorial$

### 对于似然p(x|y)

假设：**条件独立性**（特征在类别确定的情况下是独立的）
$$P(x|y)=\prod_{j=1}^mP(x_j|y)$$
- x离散：$x_j\sim Categorial$
- x连续：$x_j \sim N(\mu_j,\sigma^2_j)$